# Need to install lxml

In [92]:
import numpy as np

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import xmltodict
import os
import requests
from bs4 import BeautifulSoup
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux


FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

# Import simulation output

Before running this, I ran a sim in ecoli/experiments/new_environments.py with -n 0 to generate a simulation output. This is necessary to replicate the simulation environment, where we e.g. have dynamic demand for amino acids and nucleotides, and varying cell volumes.

In [49]:
flux_save = pd.DataFrame()

condition_df = pd.read_excel("ecoli/experiments/new_environments_test/done.xlsx")
condition_df["Add"] = condition_df["Add"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Remove"] = condition_df["Remove"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Demand"] = condition_df["Demand"].apply(lambda x: set(str(x).split(", ") if x == x else set()))

for i in range(len(condition_df)):
    time = '50'
    date = '2023-07-21'
    experiment = 'fba_new_environments'
    env_name = f'{condition_df.loc[i, "Environment"]}'
    entry = f'{experiment}_{env_name}_{time}_{date}'
    folder = f'out/fba_new_env/{entry}/'
    try:
        output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
        # output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
        output = output['agents']['0']
        fba = output['listeners']['fba_results']
        mass = output['listeners']['mass']
        bulk = pd.DataFrame(output['bulk'])
        # this is the model itself - we'll pull e.g. stoichiometry from here.
        f = open(folder + 'agent_steps.pkl', 'rb')
        agent = dill.load(f)
        f.close()

        metabolism = agent['ecoli-metabolism-redux']
        stoichiometry = metabolism.stoichiometry

        # sim_exchanges = pd.DataFrame(fba["estimated_exchange_dmdt"])
        # sim_exchanges = pd.DataFrame(sim_exchanges.loc[24, :].abs().sort_values(ascending=False))
        # if os.path.exists(f'out/new_env_analysis/{env_name}'):
        #     pass
        # else:
        #     os.mkdir(f'out/new_env_analysis/{env_name}')
        # sim_exchanges.to_csv(rf'out/new_env_analysis/{env_name}/sim_exchanges.csv')

        sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
        sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

        if i == 0:
            flux_save = sim_fluxes
        else:
            new_col = sim_fluxes.iloc[:,0].to_frame()
            flux_save[env_name] = new_col
        # sim_fluxes.to_csv(rf'out/new_env_analysis/{env_name}/sim_fluxes.csv')
        #
        # sim_catalysts = pd.DataFrame(fba["reaction_catalyst_counts"], columns=metabolism.reaction_names)
        # sim_catalysts = pd.DataFrame(sim_catalysts.loc[4, :].abs().sort_values(ascending=False))
        # sim_catalysts.to_csv(rf'out/new_env_analysis/{env_name}/sim_catalysts.csv')

    except FileNotFoundError:
        pass

In [50]:
df = pd.DataFrame(flux_save)
flux_save.to_csv(rf'out/new_env_analysis/overall_sim_fluxes.csv')

## Analysis on spreadsheet with all fluxes for all environments

In [96]:
overall_flux = pd.read_excel("out/new_env_analysis/overall_sim_fluxes.xlsx")
# print(overall_flux.head())

In [97]:
array = overall_flux.to_numpy()

for row in array:
    for i in range(len(row)):
        if i == 0 or i == len(row) - 1:
            continue
        else:
            row[i] = (row[i] - row[len(row) - 1])

In [99]:
data = array[:,1:]

reactions = list(array[:,0])
environments = list(overall_flux.columns)[1:]

heatmap_df = pd.DataFrame(data, index=reactions, columns=environments)
for env in environments:
    heatmap_df[env] = heatmap_df[env].astype(float)

heatmap_df.to_csv(rf'out/new_env_analysis/absolute_heatmap.csv')

# heatmap_df = pd.read_excel("out/new_env_analysis/absolute_heatmap.xlsx")

In [54]:
# plt.figure(figsize=(125, 125))
# sns.set(font_scale=0.5)
#
# svm = sns.heatmap(heatmap_df, annot=True, fmt=".2f", linewidths=.5, cmap="YlGnBu")
#
# plt.xlabel("Environment")
# plt.ylabel("Reaction")
# figure = svm.get_figure()
# figure.savefig('out/new_env_analysis/heatmap.png', dpi=400)

In [33]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})

<Response [200]>

In [11]:
function = "reactions-of-compound"
name = "LEU"
env = "Hydrogen_fumarate_-O2_Glc"
req_str = f"https://websvc.biocyc.org/apixml?fn={function}&id=ECOLI:{name}&detail=none&fmt=json"

r = s.get(req_str)
o = xmltodict.parse(r.content)['ptools-xml']['Reaction']
if type(o) is dict:
    o = [o]
for rxn in o:
    if type(rxn) is not str:
        to_check = []
        for reaction in reactions:
            if rxn['@frameid'] in reaction:
                to_check.append(reaction)
        for reaction in to_check:
            try:
                if heatmap_df.loc[reaction, env] != 0:
                    print(str(heatmap_df.loc[reaction, env]) + f" {reaction}")
            except KeyError:
                print("KeyError")

['BRANCHED-CHAINAMINOTRANSFERLEU-RXN (reverse)', 'BRANCHED-CHAINAMINOTRANSFERLEU-RXN__BRANCHED-CHAINAMINOTRANSFER-CPLX', 'BRANCHED-CHAINAMINOTRANSFERLEU-RXN']
KeyError
KeyError
KeyError
[]
[]
[]
['TRANS-RXN-126B']
KeyError
['TRANS-RXN0-270']
KeyError
['RXN0-6979']
KeyError
['ABC-35-RXN']
KeyError


## Tool to create graphs of upstream metabolites

In [94]:
time = '50'
date = '2023-07-21'
experiment = 'fba_new_environments'
# Change the environment name here
env_name = 'Hydrogen_fumarate_-O2_Glc'
entry = f'{experiment}_{env_name}_{time}_{date}'
folder = f'out/fba_new_env/{entry}/'

f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry

output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

metabolites_idx = {species: i for i, species in enumerate(metabolism.metabolite_names)}
reaction_catalysts = metabolism.parameters['reaction_catalysts']
metabolism.catalyst_ids = metabolism.parameters['catalyst_ids']
catalyst_idx = {catalyst: i for i, catalyst in enumerate(metabolism.catalyst_ids)}

In [75]:
class Node:
    def __init__(self, val, order):
        self.val = val
        self.nodes = []
        self.order = order

    def add_node(self, val, order):
        self.nodes.append(Node(val, order))

    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.val)+"\n"
        for child in self.nodes:
            ret += child.__repr__(level+1)
        return ret

In [88]:
index_to_met = {v: k for k, v in metabolites_idx.items()}
met_length = len(metabolism.metabolite_names)
reaction_length = np.shape(stoichiometry)[1]
met_not_include = ["ATP[c]", "WATER[c]", "PROTON[p]", "Pi[c]"]
do_not_include = []

for met in met_not_include:
    do_not_include.append(metabolites_idx[met])

def find_upstream_metabolites(metabolite):
    to_return = []
    index = metabolites_idx[metabolite]
    for k in range(reaction_length):
        if stoichiometry[index, k] > 0:
            for l in range(met_length):
                if stoichiometry[l, k] < 0:
                    if l != index and l not in do_not_include and index_to_met[l] not in to_return:
                        to_return.append(index_to_met[l])
    return to_return

In [90]:
metabolite = "LEU[c]"

root = Node(metabolite, 0)
for i in range(2):
    if i == 0:
        for met in find_upstream_metabolites(metabolite):
            root.add_node(met, 1)
    else:
        for j in range(len(root.nodes)):
            for met in find_upstream_metabolites(root.nodes[j].val):
                if met not in root.nodes[j].nodes:
                    root.nodes[j].add_node(met, 2)

print(root)


'LEU[c]'
	'LEU[p]'
		'LEU[c]'
		'LEU[e]'
	'2K-4CH3-PENTANOATE[c]'
		'2-KETOGLUTARATE[c]'
		'LEU[c]'
		'2-D-THREO-HYDROXY-3-CARBOXY-ISOCAPROATE[c]'
		'NAD[c]'
		'CPD-7100[c]'
		'PROTON[c]'
	'GLT[c]'
		'2-KETOGLUTARATE[c]'
		'Aromatic-Amino-Acids[c]'
		'PUTRESCINE[c]'
		'5-OXOPROLINE[c]'
		'GLT[p]'
		'L-ALPHA-ALANINE[c]'
		'CHORISMATE[c]'
		'GLN[c]'
		'L-ASPARTATE[c]'
		'ILE[c]'
		'LEU[c]'
		'VAL[c]'
		'HCO3[c]'
		'UTP[c]'
		'CYS[c]'
		'5-P-RIBOSYL-N-FORMYLGLYCINEAMIDE[c]'
		'4-AMINO-BUTYRATE[c]'
		'NADPH[c]'
		'PROTON[c]'
		'PHOSPHORIBULOSYL-FORMIMINO-AICAR-P[c]'
		'AMMONIUM[c]'
		'D-GLT[c]'
		'XANTHOSINE-5-PHOSPHATE[c]'
		'L-HISTIDINOL-P[c]'
		'FRUCTOSE-6P[c]'
		'ACETYL-GLU[c]'
		'CO-A[c]'
		'DEAMIDO-NAD[c]'
		'PRPP[c]'
		'3-P-SERINE[c]'
		'4-PHOSPHONOOXY-THREONINE[c]'
		'L-DELTA1-PYRROLINE_5-CARBOXYLATE[c]'
		'NAD[c]'
		'MET[c]'
		'PHE[c]'
		'L-GLUTAMATE_GAMMA-SEMIALDEHYDE[c]'
		'CPD-14736[c]'
		'UDP-4-AMINO-4-DEOXY-L-ARABINOSE[c]'
		'CPD-9000[c]'
		'CPD0-889[c]'
		'CPD0-1445[c]'
		'C

## Pathway visualization of changing fluxes

                                                            24
R601-RXN-FUM/REDUCED-MENAQUINONE//SUC/CPD-9728.38.  20133261.0
RXN0-5256-CPD-9728/PROTON/HYDROGEN-MOLECULE//RE...  20101063.0
TRANS-RXN-106                                       20033460.0
TRANS-RXN-145                                       10477226.0
ATPSYN-RXN (reverse)                                 9599983.0
...                                                        ...
RXN-21538                                                  0.0
RXN-21533-CPD-23688/ATP//CPD-23428/ADP/PROTON.36.          0.0
RXN-21533-CPD-23431/ATP//CPD-23690/ADP/PROTON.36.          0.0
RXN-21533-CPD-23431/ATP//CPD-23428/ADP/PROTON.36.          0.0
RXN0-7200-NMNH//NMNH.11. (reverse)                         0.0

[7504 rows x 1 columns]
